In [1]:
import yaml
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from src.models.classification import Classification
from src.models.utils import train_test_splitt

In [2]:
# read config
with open('config.yml', 'r') as file:
    config=yaml.load(file, Loader= yaml.SafeLoader)
del file

In [3]:
# load data
df = pd.read_csv(config['data_loader']['path'])

df.head()

,id,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
0,0,-0.260648,-0.469648,2.496266,-0.083724,0.129681,0.732898,0.519014,-0.130006,0.727159,...,-0.110552,0.217606,-0.134794,0.165959,0.126280,-0.434824,-0.081230,-0.151045,17982.10,0
1,1,0.985100,-0.356045,0.558056,-0.429654,0.277140,0.428605,0.406466,-0.133118,0.347452,...,-0.194936,-0.605761,0.079469,-0.577395,0.190090,0.296503,-0.248052,-0.064512,6531.37,0
2,2,-0.260272,-0.949385,1.728538,-0.457986,0.074062,1.419481,0.743511,-0.095576,-0.261297,...,-0.005020,0.702906,0.945045,-1.154666,-0.605564,-0.312895,-0.300258,-0.244718,2513.54,0
3,3,-0.152152,-0.508959,1.746840,-1.090178,0.249486,1.143312,0.518269,-0.065130,-0.205698,...,-0.146927,-0.038212,-0.214048,-1.893131,1.003963,-0.515950,-0.165316,0.048424,5384.44,0
4,4,-0.206820,-0.165280,1.527053,-0.448293,0.106125,0.530549,0.658849,-0.212660,1.049921,...,-0.106984,0.729727,-0.161666,0.312561,-0.414116,1.071126,0.023712,0.419117,14278.97,0


In [ ]:
# Nan Values
display(df.isna().sum()[df.isna().sum()>0])

# class distribution
display(np.unique(df['Class'], return_counts=True))

In [ ]:
# features distribution
display(df.describe().round(1))

# all anonimized columns - V1, ..., 28, follow a standard normal distribution
df[df.columns[1:-2]].describe().round(1).iloc[[1,2],:].T.drop_duplicates()

In [ ]:
# amount distribution per class label
plt.figure(figsize=(10, 6))
sns.kdeplot(data=df, x='Amount', hue='Class', fill=True, common_norm=True)
plt.title('Kernel Density Estimate of Amount by Class Label')
plt.xlabel('Amount')
plt.ylabel('Density')
plt.show()

In [11]:
d = train_test_splitt(
    X=df.iloc[:,1:-1]
    , y=df['Class']
    , is_test_split=True
    , **{
        'test_size': config['model']['test_set_size']
        , 'random_state': 123
        , 'shuffle': True
        }
    )

In [23]:
np.unique(d['y_train'], return_counts=True), d['y_train'].shape[0] / df.shape[0] *100

((array([0, 1]), array([270197, 270001])), 94.9999120693597)

In [26]:
clf = Classification(
    algorithm='logistic_regression'
    )
clf.fit(X=d['X_train'], y=d['y_train'])

C:\Users\gonpr\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\linear_model\_logistic.py:465: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. OF ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [28]:
y_pred = clf.predict(X=d['X_test'])

In [29]:
from sklearn.metrics import accuracy_score
accuracy_score(d['y_test'], y_pred)

0.9550506471581317

In [31]:
from sklearn.metrics import precision_score, recall_score, f1_score
precision_score(d['y_test'], y_pred), recall_score(d['y_test'], y_pred), f1_score(d['y_test'], y_pred)

(0.9772294625860302, 0.932443761352522, 0.9543114543114544)